In [1]:
import pandas as pd
import numpy as np
import polars as pl
from pathlib import Path

# Setup paths
root = Path().resolve().parent
cboe_path = root / 'data' / 'cboe'

# Define the years you want to process
years = range(2015, 2025)  # 2015 to 2024

# List to store daily data from each year
all_daily_data = []
previous_year_last_price = None  # Track last price from previous year

print("Processing data from 2015 to 2024...")

for year in years:
    year_path = cboe_path / str(year)

    # Check if the year directory exists
    if not year_path.exists():
        print(f"Warning: Directory for {year} not found, skipping...")
        continue

    print(f"\nProcessing year {year}...")

    try:
        # Load data with Polars and convert to pandas
        lazy_df = pl.scan_parquet(year_path / '*.gzip.parquet')
        df = lazy_df.collect().to_pandas()

        print(f"  Loaded {len(df)} rows for {year}")

        # Step 1: Get underlying prices at each unique datetime
        underlying_prices = (
            df[['quote_datetime', 'active_underlying_price']]
            .drop_duplicates(subset=['quote_datetime'])
            .sort_values('quote_datetime')
            .reset_index(drop=True)
        )

        # Free up memory
        del df

        # Step 2: Calculate intraday log returns
        underlying_prices['log_return'] = (
            np.log(underlying_prices['active_underlying_price']) -
            np.log(underlying_prices['active_underlying_price'].shift(1))
        )

        # Calculate squared log returns for realized variance
        underlying_prices['squared_log_return'] = underlying_prices['log_return'] ** 2

        # Step 3: Extract date for grouping
        underlying_prices['date'] = pd.to_datetime(underlying_prices['quote_datetime']).dt.date

        # Step 4: Calculate DAILY realized variance
        daily_data = underlying_prices.groupby('date').agg({
            'squared_log_return': 'sum',
            'quote_datetime': 'count'
        }).reset_index()

        daily_data.columns = ['date', 'sum_squared_returns', 'n_obs']
        daily_data['realized_variance'] = daily_data['sum_squared_returns'] * (100 ** 2)

        # Step 5: Calculate DAILY close-to-close log returns
        daily_close_prices = underlying_prices.groupby('date').agg({
            'active_underlying_price': 'last'
        }).reset_index()

        daily_close_prices.columns = ['date', 'close_price']

        # Calculate daily returns, handling the boundary with previous year
        daily_close_prices['daily_log_return'] = (
            np.log(daily_close_prices['close_price']) -
            np.log(daily_close_prices['close_price'].shift(1))
        )

        # Fill the first day's return using previous year's last price
        if previous_year_last_price is not None:
            first_day_return = (
                np.log(daily_close_prices.loc[0, 'close_price']) -
                np.log(previous_year_last_price)
            )
            daily_close_prices.loc[0, 'daily_log_return'] = first_day_return

        # Store last price for next year
        previous_year_last_price = daily_close_prices.loc[len(daily_close_prices)-1, 'close_price']

        daily_close_prices['daily_log_return_pct'] = daily_close_prices['daily_log_return'] * 100

        # Step 6: Merge daily returns with realized variance
        daily_data = daily_data.merge(
            daily_close_prices[['date', 'daily_log_return_pct']],
            on='date',
            how='left'
        )

        # Add year column for reference
        daily_data['year'] = year

        # Append to list
        all_daily_data.append(daily_data)

        print(f"  Processed {len(daily_data)} daily observations for {year}")

        # Free up memory
        del underlying_prices, daily_close_prices

    except Exception as e:
        print(f"  Error processing {year}: {str(e)}")
        continue

# Concatenate all years
print("\nCombining all years...")
combined_daily_data = pd.concat(all_daily_data, ignore_index=True)

# Free up memory
del all_daily_data

# Sort by date
combined_daily_data = combined_daily_data.sort_values('date').reset_index(drop=True)

print(f"\nTotal daily observations across all years: {len(combined_daily_data)}")
print(f"Date range: {combined_daily_data['date'].min()} to {combined_daily_data['date'].max()}")

# Step 7: Demean the daily returns across ALL years (skip NaN if first row of first year)
mean_return = combined_daily_data['daily_log_return_pct'].mean()
combined_daily_data['demeaned_log_return'] = combined_daily_data['daily_log_return_pct'] - mean_return

print(f"Mean return across all years: {mean_return:.6f}%")

# Step 8: Create forward-looking realized variance
combined_daily_data['forward_realized_variance'] = combined_daily_data['realized_variance'].shift(-1)

# Step 9: Remove first row if it has NaN return (first day of 2015) and last row (no forward RV)
if pd.isna(combined_daily_data.loc[0, 'daily_log_return_pct']):
    combined_daily_data = combined_daily_data.iloc[1:-1].reset_index(drop=True)
else:
    combined_daily_data = combined_daily_data.iloc[:-1].reset_index(drop=True)

# Step 10: Save to CSV
output_path = root / 'data' / 'processed' / 'daily_data_2015_2024.csv'
output_path.parent.mkdir(parents=True, exist_ok=True)
combined_daily_data.to_csv(output_path, index=False)

print(f"\nData saved to: {output_path}")
print(f"Final shape: {combined_daily_data.shape}")
print(f"\nChecking for NaN values:")
print(combined_daily_data.isnull().sum())
print("\nSample of final data:")
print(combined_daily_data.head())

Processing data from 2015 to 2024...

Processing year 2015...
  Loaded 15396240 rows for 2015
  Processed 252 daily observations for 2015

Processing year 2016...
  Loaded 14305464 rows for 2016
  Processed 252 daily observations for 2016

Processing year 2017...
  Loaded 20537088 rows for 2017
  Processed 251 daily observations for 2017

Processing year 2018...
  Loaded 29011440 rows for 2018
  Processed 251 daily observations for 2018

Processing year 2019...
  Loaded 30774460 rows for 2019
  Processed 252 daily observations for 2019

Processing year 2020...
  Loaded 35945472 rows for 2020
  Processed 253 daily observations for 2020

Processing year 2021...
  Loaded 41214126 rows for 2021
  Processed 252 daily observations for 2021

Processing year 2022...
  Loaded 38898714 rows for 2022
  Processed 251 daily observations for 2022

Processing year 2023...
  Loaded 38802410 rows for 2023
  Processed 250 daily observations for 2023

Processing year 2024...
  Loaded 44884476 rows for 20

In [3]:
output_path

PosixPath('/Users/gregruyoga/gmoneycodes/gmsm/gmsm/data/processed/daily_data_2015_2024.csv')

In [2]:
pd.read_csv(output_path)

,date,sum_squared_returns,n_obs,realized_variance,daily_log_return_pct,year,demeaned_log_return,forward_realized_variance
0,2015-01-05,0.000101,12,1.013478,-1.832404,2015,-1.875813,2.209438
1,2015-01-06,0.000221,12,2.209438,-0.921207,2015,-0.964617,0.849982
2,2015-01-07,0.000085,12,0.849982,1.168826,2015,1.125416,0.916830
3,2015-01-08,0.000092,12,0.916830,1.771187,2015,1.727778,0.360881
4,2015-01-09,0.000036,12,0.360881,-0.813199,2015,-0.856608,1.490471
...,...,...,...,...,...,...,...,...
2493,2024-11-29,0.000011,6,0.110078,0.583303,2024,0.539893,0.097063
2494,2024-12-02,0.000010,12,0.097063,0.215456,2024,0.172047,0.052435
2495,2024-12-03,0.000005,12,0.052435,0.039184,2024,-0.004225,0.145778
2496,2024-12-04,0.000015,12,0.145778,0.605235,2024,0.561825,0.092116
